# Rakentamassa tekoälyagentteja pysyvällä muistilla käyttäen Cognee

Tämä muistikirja esittelee, kuinka rakentaa älykkäitä tekoälyagentteja kehittyneillä muistitoiminnoilla käyttäen [**cognee**](https://www.cognee.ai/) - avoimen lähdekoodin tekoälymuistia, joka yhdistää tietämyskartat, semanttisen haun ja istunnonhallinnan luodakseen kontekstia ymmärtäviä tekoälyjärjestelmiä.

## 🎯 Oppimistavoitteet

Tämän opetusohjelman lopussa ymmärrät, kuinka:
- **Rakentaa tietämyskarttoja upotusten avulla**: Muuntaa jäsentämätön teksti rakenteelliseksi, kyseltäväksi tiedoksi
- **Toteuttaa istuntomuisti**: Luoda monivaiheisia keskusteluja automaattisella kontekstin säilytyksellä
- **Säilyttää keskusteluja**: Tallentaa tarvittaessa tärkeitä vuorovaikutuksia pitkäaikaiseen muistiin tulevaa käyttöä varten
- **Kysellä luonnollisella kielellä**: Käyttää ja hyödyntää historiallista kontekstia uusissa keskusteluissa
- **Visualisoida muisti**: Tutkia agenttisi tietämyskartan suhteita


## 🏗️ Mitä tulet rakentamaan

Tässä opetusohjelmassa luomme **Koodausavustajan**, jolla on pysyvä muisti ja joka:

### 1. **Tietopohjan rakentaminen**
   - Ottaa vastaan kehittäjän profiili- ja asiantuntijatietoja
   - Käsittelee Python-ohjelmoinnin periaatteita ja parhaita käytäntöjä
   - Tallentaa historialliset keskustelut kehittäjien ja tekoälyavustajien välillä

### 2. **Istuntokohtaiset keskustelut**
   - Säilyttää kontekstin useiden kysymysten välillä samassa istunnossa
   - Tallentaa automaattisesti jokaisen kysymys/vastaus-parin tehokasta hakua varten
   - Tarjoaa johdonmukaisia ja kontekstuaalisia vastauksia keskusteluhistorian perusteella

### 3. **Pitkäaikainen muisti**
   - Tallentaa tärkeät keskustelut pitkäaikaiseen muistiin
   - Hakee olennaisia muistoja tietopohjasta ja aiemmista istunnoista uusien vuorovaikutusten tueksi
   - Rakentaa kasvavan tietopohjan, joka paranee ajan myötä

### 4. **Älykäs muistinhaku**
   - Käyttää graafitietoista semanttista hakua löytääkseen olennaista tietoa kaikesta tallennetusta tiedosta
   - Suodattaa hakuja tietoryhmien mukaan (kehittäjätiedot vs. periaatteet)
   - Yhdistää useita tietolähteitä tarjotakseen kattavia vastauksia


## 📋 Esivaatimukset ja asennus

### Järjestelmävaatimukset

Ennen aloittamista varmista, että sinulla on:

1. **Python-ympäristö**
   - Python 3.9 tai uudempi
   - Virtuaalinen ympäristö (suositeltu)
   
2. **Redis-välimuisti** (vaaditaan istunnon hallintaan)
   - Paikallinen Redis: `docker run -d -p 6379:6379 redis`
   - Tai käytä hallinnoitua Redis-palvelua
   
3. **LLM API -pääsy**
   - OpenAI API -avain tai muiden palveluntarjoajien avain (katso [dokumentaatio](https://docs.cognee.ai/setup-configuration/llm-providers))

4. **Tietokantakonfiguraatio**
   - Oletuksena ei vaadita konfiguraatiota. Cognee käyttää tiedostopohjaisia tietokantoja (LanceDB ja Kuzu)
   - Vaihtoehtoisesti voit ottaa käyttöön Azure AI Searchin vektorivarastona (katso [dokumentaatio](https://github.com/topoteretes/cognee-community/tree/main/packages/vector/azureaisearch))

### Ympäristön konfigurointi

Luo `.env`-tiedosto projektikansioosi seuraavilla muuttujilla:

```ini
# LLM Configuration (Required)
LLM_API_KEY=your-openai-api-key-here

# Cache Configuration (Required for Sessions)
CACHING=true  # Must be enabled for session history

```


## 🏛️ Cogneen muistirakenteen ymmärtäminen

### Miten Cognee toimii

Cognee tarjoaa kehittyneen muistijärjestelmän, joka ylittää yksinkertaisen avain-arvo-tallennuksen:

```
┌──────────────────────────┐
│      30+ data sources    │
└───────────┬──────────────┘
            │
            ▼
┌──────────────────────────────────────────┐
│  Dynamically evolving memory layers      │
│                                          │
│  ┌────────────────────────────────────┐  │
│  │ Knowledge Graph in Graph Database  │  │
│  └────────────────────────────────────┘  │
│  ┌────────────────────────────────────┐  │
│  │ Embeddings in Vector Store         │  │
│  │   (e.g., Azure AI Search)          │  │
│  └────────────────────────────────────┘  │
└───────────┬──────────────────────────────┘
            │                      ▲   
            ▼                      │(optional)
┌────────────────┐           ┌────────────────┐
│     cognee     │(optional) │ Cognee Session │
│    retrievers  │──────────▶│     Cache      │
│                │           │    (Redis)     │
└───────┬────────┘           └────────────────┘
        ▲
        │
┌──────────────────────────┐
│          Agents          │
└──────────────────────────┘

```

### Keskeiset komponentit:

1. **Tietämyskartta**: Tallentaa entiteetit, suhteet ja semanttiset yhteydet
2. **Vektoriesitykset**: Mahdollistaa semanttisen haun kaikesta tallennetusta tiedosta
3. **Istuntovälimuisti**: Säilyttää keskustelukontekstin istuntojen sisällä ja välillä
4. **NodeSets**: Järjestää tiedot loogisiin kategorioihin kohdennettua hakua varten

### Muistityypit tässä opetusohjelmassa:

- **Pysyvä muisti**: Pitkäaikaistallennus tietämyskartassa
- **Istuntomuisti**: Väliaikainen keskustelukonteksti Redis-välimuistissa
- **Semanttinen muisti**: Vektoreihin perustuva samankaltaisuushaku kaikesta datasta


## 📦 Asenna tarvittavat paketit

Asenna Cognee Redis-tuen kanssa istunnon hallintaa varten:


In [ ]:
!pip install --quiet "cognee[redis]==0.4.0"

## 🔧 Alusta ympäristö ja lataa kirjastot

Varmista, että:
1. Redis on käynnissä (esim. Dockerin kautta: `docker run -d -p 6379:6379 redis`)
2. Ympäristömuuttujat on asetettu ennen välimuistimoduulien tuontia
3. Jos tarpeen, käynnistä ydin uudelleen ja suorita solut järjestyksessä

Seuraava solu:
1. Lataa ympäristömuuttujat tiedostosta `.env`
2. Määrittää Cogneen LLM-asetuksillasi
3. Ota välimuisti käyttöön istunnon hallintaa varten
4. Varmistaa, että kaikki komponentit ovat asianmukaisesti yhdistettyjä


In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# cognee Configuration
os.environ["LLM_API_KEY"] = os.getenv("LLM_API_KEY", None)
os.environ["CACHING"] = os.getenv("CACHING", "true")


import cognee

print(f"Cognee version: {cognee.__version__}")
print(f"CACHING: {os.environ.get('CACHING')}")
print(f"LLM_API_KEY: {os.environ.get('LLM_API_KEY')}")

## 📁 Määritä tallennuskansiot

Cognee käyttää kahta erillistä hakemistoa toiminnassaan:
- **Data Root**: Tallentaa syötetyt dokumentit ja käsitellyn datan
- **System Root**: Sisältää tietämyskartan tietokannan ja järjestelmän metatiedot

Luomme erilliset hakemistot tätä opasta varten seuraavasti:


In [ ]:
DATA_ROOT = Path('.data_storage').resolve()
SYSTEM_ROOT = Path('.cognee_system').resolve()

DATA_ROOT.mkdir(parents=True, exist_ok=True)
SYSTEM_ROOT.mkdir(parents=True, exist_ok=True)

cognee.config.data_root_directory(str(DATA_ROOT))
cognee.config.system_root_directory(str(SYSTEM_ROOT))

print(f"Data root: {DATA_ROOT}")
print(f"System root: {SYSTEM_ROOT}")

## 🧹 Nollaa muistintila

Ennen kuin aloitamme muistijärjestelmän rakentamisen, varmistetaan, että aloitamme puhtaalta pöydältä.

> 💡 **Vinkki**: Voit ohittaa tämän vaiheen, jos haluat säilyttää aiemmista ajoista peräisin olevat muistot, kun käytät tätä muistikirjaa myöhemmin.


In [ ]:
await cognee.prune.prune_data()
await cognee.prune.prune_system(metadata=True)
print('Cleared previous Cognee state.')

## 📚 Osa 1: Tietopohjan rakentaminen

### Tietolähteet kehittäjäavustajallemme

Luodaksemme kattavan tietopohjan, hyödynnämme kolmenlaisia tietoja:

1. **Kehittäjäprofiili**: Henkilökohtainen asiantuntemus ja tekninen tausta  
2. **Pythonin parhaat käytännöt**: Pythonin Zen ja käytännön ohjeet  
3. **Aiemmat keskustelut**: Kehittäjien ja tekoälyavustajien väliset kysymys-vastaus-sessiot  

Tämä monipuolinen tieto mahdollistaa agenttimme:  
- Ymmärtää käyttäjän teknisen taustan  
- Soveltaa parhaita käytäntöjä suosituksissaan  
- Oppia aiemmista onnistuneista vuorovaikutuksista  


In [ ]:
developer_intro = (
  "Hi, I'm an AI/Backend engineer. "
  "I build FastAPI services with Pydantic, heavy asyncio/aiohttp pipelines, "
  "and production testing via pytest-asyncio. "
  "I've shipped low-latency APIs on AWS, Azure, and GoogleCloud."
)

python_zen_principles = (
  """
    # The Zen of Python: Practical Guide

    ## Overview
    Use these principles as a checklist during design, coding, and reviews.

    ## Key Principles With Guidance

    ### 1. Beautiful is better than ugly
    Prefer descriptive names, clear structure, and consistent formatting.

    ### 2. Explicit is better than implicit
    Be clear about behavior, imports, and types.
    ```python
    from datetime import datetime, timedelta

    def get_future_date(days_ahead: int) -> datetime:
        return datetime.now() + timedelta(days=days_ahead)
    ```

    ### 3. Simple is better than complex
    Choose straightforward solutions first.

    ### 4. Complex is better than complicated
    When complexity is needed, organize it with clear abstractions.

    ### 5. Flat is better than nested
    Use early returns to reduce indentation.

    ## Modern Python Tie-ins
    - Type hints reinforce explicitness
    - Context managers enforce safe resource handling
    - Dataclasses improve readability for data containers

    ## Quick Review Checklist
    - Is it readable and explicit?
    - Is this the simplest working solution?
    - Are errors explicit and logged?
    - Are modules/namespaces used appropriately?
  """
)

human_agent_conversations = (
  """
  "conversations": [
      {
        "id": "conv_001",
        "timestamp": "2024-01-15T10:30:00Z",
        "topic": "async/await patterns",
        "user_query": "I'm building a web scraper that needs to handle thousands of URLs concurrently. What's the best way to structure this with asyncio?",
        "assistant_response": "Use asyncio with aiohttp, a semaphore to cap concurrency, TCPConnector for connection pooling, context managers for session lifecycle, and robust exception handling for failed requests.",
        "code_context": {
          "file": "scraper.py",
          "language": "python",
          "patterns_discussed": ["async/await", "context_managers", "semaphores", "aiohttp", "error_handling"]
        },
        "follow_up_questions": [
          "How do I add retry logic for failed requests?",
          "What's the best way to parse the scraped HTML content?"
        ]
      },
      {
        "id": "conv_002",
        "timestamp": "2024-01-16T14:20:00Z",
        "topic": "dataclass vs pydantic",
        "user_query": "When should I use dataclasses vs Pydantic models? I'm building an API and need to handle user input validation.",
        "assistant_response": "For API input/output, prefer Pydantic: it provides runtime validation, type coercion, JSON serialization, enums for roles, field constraints, and custom validators; integrates cleanly with FastAPI for automatic request validation and error reporting.",
        "code_context": {
          "file": "models.py",
          "language": "python",
          "patterns_discussed": ["pydantic", "dataclasses", "validation", "fastapi", "type_hints", "enums"]
        },
        "follow_up_questions": [
          "How do I handle nested validation with Pydantic?",
          "Can I use Pydantic with SQLAlchemy models?"
        ]
      },
      {
        "id": "conv_003",
        "timestamp": "2024-01-17T09:15:00Z",
        "topic": "testing patterns",
        "user_query": "I'm struggling with testing async code and database interactions. What's the best approach for pytest with async functions?",
        "assistant_response": "Recommended using pytest-asyncio, async fixtures, and an isolated test database or mocks to reliably test async functions and database interactions in FastAPI.",
        "code_context": {
          "file": "test_users.py",
          "language": "python",
          "patterns_discussed": ["pytest", "async_testing", "fixtures", "mocking", "database_testing", "fastapi_testing"]
        },
        "follow_up_questions": [
          "How do I test WebSocket connections?",
          "What's the best way to test database migrations?"
        ]
      },
      {
        "id": "conv_004",
        "timestamp": "2024-01-18T16:45:00Z",
        "topic": "performance optimization",
        "user_query": "My FastAPI app is getting slow with large datasets. How can I optimize database queries and response times?",
        "assistant_response": "Suggested optimizing database queries (indexes, pagination, selecting only needed columns), adding caching, streaming or chunked responses for large datasets, background tasks for heavy work, and monitoring to find bottlenecks.",
        "code_context": {
          "file": "optimizations.py",
          "language": "python",
          "patterns_discussed": ["performance_optimization", "caching", "database_optimization", "async_patterns", "monitoring"]
        },
        "follow_up_questions": [
          "How do I implement database connection pooling properly?",
          "What's the best way to handle memory usage with large datasets?"
        ]
      },
      {
        "id": "conv_005",
        "timestamp": "2024-01-19T11:30:00Z",
        "topic": "error handling and logging",
        "user_query": "I need to implement proper error handling and logging across my Python application. What's the best approach for production-ready error management?",
        "assistant_response": "Proposed centralized error handling with custom exceptions, structured logging, FastAPI middleware or decorators, and integration points for external monitoring/alerting tools.",
        "code_context": {
          "file": "error_handling.py",
          "language": "python",
          "patterns_discussed": ["error_handling", "logging", "exceptions", "middleware", "decorators", "fastapi"]
        },
        "follow_up_questions": [
          "How do I integrate this with external monitoring tools like Sentry?",
          "What's the best way to handle errors in background tasks?"
        ]
      }
    ],
    "metadata": {
      "total_conversations": 5,
      "date_range": "2024-01-15 to 2024-01-19",
      "topics_covered": [
        "async/await patterns",
        "dataclass vs pydantic",
        "testing patterns",
        "performance optimization",
        "error handling and logging"
      ],
      "code_patterns_discussed": [
        "asyncio", "aiohttp", "semaphores", "context_managers",
        "pydantic", "fastapi", "type_hints", "validation",
        "pytest", "async_testing", "fixtures", "mocking",
        "performance_optimization", "caching", "database_optimization",
        "error_handling", "logging", "exceptions", "middleware"
      ],
      "difficulty_levels": {
        "beginner": 1,
        "intermediate": 2,
        "advanced": 2
      }
    }
  """
)

## 🔄 Muunna data tietämysgrafiksi

Nyt muutamme raakatekstimme jäsennellyksi muistiksi. Tämä prosessi:

1. **Lisää dataa NodeSeteihin**: Järjestää tiedot loogisiin kategorioihin
   - `developer_data`: Kehittäjäprofiili ja keskustelut
   - `principles_data`: Pythonin parhaat käytännöt ja ohjeet

2. **Suorittaa Cognify Pipeline**: Erottaa entiteetit, suhteet ja luo upotuksia
   - Tunnistaa keskeiset käsitteet
   - Luo semanttisia yhteyksiä liittyvän tiedon välille
   - Generoi vektoriupotuksia

Tämä voi kestää hetken, kun LLM käsittelee tekstiä ja rakentaa graafirakenteen:


In [ ]:
await cognee.add(developer_intro, node_set=["developer_data"])
await cognee.add(human_agent_conversations, node_set=["developer_data"])
await cognee.add(python_zen_principles, node_set=["principles_data"])

await cognee.cognify()

## 📊 Visualisoi tietämyskartta

Tutkitaan tietämyskarttamme rakennetta. Visualisointi näyttää:
- **Solmut**: Tekstistä poimitut entiteetit (käsitteet, teknologiat, henkilöt)
- **Reunat**: Entiteettien väliset suhteet ja yhteydet
- **Klusterit**: Semanttisen samankaltaisuuden perusteella ryhmitellyt liittyvät käsitteet

Avaa luotu HTML-tiedosto selaimessasi tutkiaksesi karttaa interaktiivisesti:


In [ ]:
from cognee import visualize_graph
await visualize_graph('./visualization_1.html')

## 🧠 Rikasta muistia Memify-toiminnolla

`memify()`-funktio analysoi tietämyskartan ja luo älykkäitä sääntöjä datasta. Tämä prosessi:
- Tunnistaa kaavoja ja parhaita käytäntöjä
- Luo sisältöön perustuvia toimintaohjeita
- Määrittää yhteyksiä eri tietämysalueiden välillä

Nämä säännöt auttavat agenttia tekemään paremmin perusteltuja päätöksiä kysymyksiin vastatessaan. Toisen visualisoinnin tallentaminen auttaa sinua vertaamaan, kuinka kartta tiivistyy rikastamisen jälkeen.


In [ ]:
await cognee.memify()

await visualize_graph('./visualization_2.html')

## 🔍 Osa 2: Älykäs tiedonhaku

### Demonstraatio 1: Tiedon yhdistäminen eri dokumenteista

Nyt kun tietämyskartta on luotu, testataan, miten Cognee yhdistää tietoa useista lähteistä vastatakseen monimutkaisiin kysymyksiin.

Ensimmäinen kysely havainnollistaa:
- **Semanttinen ymmärrys**: Löytää olennaisia käsitteitä, vaikka niitä ei mainittaisi suoraan
- **Ristiinviittaus**: Yhdistää kehittäjäprofiilin Python-periaatteisiin
- **Kontekstuaalinen päättely**: Soveltaa parhaita käytäntöjä tiettyihin toteutuksiin

### Demonstraatio 2: Suodatettu haku NodeSetsin avulla

Toinen kysely näyttää, miten kohdistaa tietämyskartan tiettyihin osajoukkoihin:
- Käyttää `node_name`-parametria etsiäkseen vain `principles_data`-alueelta
- Tarjoaa keskittyneitä vastauksia tietystä tietämysalueesta
- Hyödyllinen, kun tarvitset alakohtaista tietoa


In [ ]:
# demonstrate cross-document knowledge retrieval from multiple data sources
from cognee.modules.search.types import SearchType

results = await cognee.search(
    query_text="How does my AsyncWebScraper implementation align with Python's design principles?",
    query_type=SearchType.GRAPH_COMPLETION,
)
print("Python Pattern Analysis:", results)

# demonstrate filtered search using NodeSet to query only specific subsets of memory
from cognee.modules.engine.models.node_set import NodeSet
results = await cognee.search(
    query_text="How should variables be named?",
    query_type=SearchType.GRAPH_COMPLETION,
    node_type=NodeSet,
    node_name=["principles_data"],
)
print("Filtered search result:", results)

## 🔐 Osa 3: Istunnon hallinnan asennus

### Keskustelumuistin käyttöönotto

Istunnon hallinta on tärkeää kontekstin säilyttämiseksi useiden vuorovaikutusten aikana. Tässä teemme seuraavat vaiheet:

1. **Käyttäjäkontekstin alustaminen**: Luo tai hae käyttäjäprofiili istunnon seurantaa varten
2. **Välimuistimoottorin määrittäminen**: Yhdistä Redis-palvelimeen keskusteluhistorian tallentamista varten
3. **Istuntomuuttujien käyttöönotto**: Määritä kontekstimuuttujat, jotka säilyvät kyselyiden välillä

> ⚠️ **Tärkeää**: Tämä edellyttää, että Redis on käynnissä ja ympäristössäsi on asetettu `CACHING=true`


In [ ]:
from cognee.modules.users.methods import get_default_user
from cognee.context_global_variables import set_session_user_context_variable 
from cognee.infrastructure.databases.cache import get_cache_engine

user = await get_default_user()
await set_session_user_context_variable(user)
print(f"Using user id: {getattr(user, 'id', 'unknown')}")

cache_engine = get_cache_engine()
if cache_engine is None:
    raise RuntimeError('Cache engine is not available. Double-check your cache configuration.')
print('Session cache is ready.')


## 🛠️ Apufunktio: Tarkastele istuntotietoja

Tämä apufunktio mahdollistaa Redisissä tallennetun keskusteluhistorian tarkastelun. Se on hyödyllinen seuraaviin tarkoituksiin:
- Istunnon hallinnan virheiden selvittäminen
- Varmistaminen, että keskustelut tallentuvat välimuistiin
- Ymmärtäminen, mitä kontekstia agentille on saatavilla


In [ ]:
async def show_history(session_id: str) -> None:
    # Let's check the cache directly
    cache_engine = get_cache_engine()
    if cache_engine:
        # Try to get history directly from cache
        user_id = str(user.id) if hasattr(user, 'id') else None
        if user_id:
            history_entries = await cache_engine.get_latest_qa(user_id, session_id, last_n=10)
            print(f"\nDirect cache query for user_id={user_id}, session_id={session_id}:")
            print(f"Found {len(history_entries)} entries")
            if history_entries:
                for i, entry in enumerate(history_entries, 1):
                    print(f"\nEntry {i}:")
                    print(f"  Question: {entry.get('question', 'N/A')[:100]}...")
                    print(f"  Answer: {entry.get('answer', 'N/A')[:100]}...")
        else:
            print("No user_id available")


## Istunto 1: Async Support Lab — Ensimmäinen kysymys

Aloita `async-support-lab`-istunto kysymällä telemetria-ystävällisiä asyncio-malleja massiiviselle web-skraperille. Graafi tuntee jo asyncio:n, aiohttp:n ja seurantakäytännöt, joten vastauksen tulisi heijastaa aiempia keskusteluja samalla kun se mukautetaan uuteen kysymykseen.


In [ ]:
session_1 = "async-support-lab"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="I'm building a web scraper that hits thousands of URLs concurrently. What's a reliable asyncio pattern with telemetry?",
    session_id=session_1
)

## Tarkastele istunnon 1 muistia ensimmäisen vaihdon jälkeen

Komennon `show_history(session_1)` suorittaminen heti alkuperäisen kysymyksen jälkeen vahvistaa, että Cognee tallensi sekä kehotteen että vastauksen Redis-tietokantaan. Sinun pitäisi nähdä yksi merkintä, jossa on samanaikaisuuden ohjeistus.


In [ ]:
await show_history(session_1)

## Istunto 1: Jatkoa tietomalleihin

Seuraavaksi kysymme: "Milloin minun pitäisi valita dataclasses verrattuna Pydanticiin?" käyttäen samaa istuntotunnusta. Cogneen tulisi yhdistää Pythonin periaatteet sekä aiemmat FastAPI-keskustelut tarjotakseen hienovaraista neuvontaa—osoittaen, että konteksti säilyy nimetyssä istunnossa.


In [ ]:
result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="When should I pick dataclasses versus Pydantic for this work?",
    session_id=session_1
)

## Vahvista, että istunnon 1 historia sisältää molemmat vuorot

Toinen `show_history(session_1)`-kutsu pitäisi näyttää kaksi kysymys-vastausmerkintää. Tämä vastaa Mem0-laboratorion "muistin toisto" -vaihetta ja todistaa, että lisävuorot laajentavat samaa keskusteluhistoriaa.


In [ ]:
await show_history(session_1)

## Istunto 2: Suunnittelukatsauksen ketju — Uusi istunto

Näyttääksemme ketjujen eristyksen käynnistämme `design-review-session` ja pyydämme lokiohjeita tapauskatsauksia varten. Vaikka taustalla oleva tietopohja on sama, uusi istuntotunnus pitää keskustelut erillään.


In [ ]:
session_2 = "design-review-session"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="We're drafting logging guidance for incident reviews. Capture the key principles please.",
    session_id=session_2
)

## Katsaus Istuntoon 2 Historia

`show_history(session_2)` pitäisi näyttää vain suunnitteluarvion kehotteen/vastauksen parin. Vertaa sitä Istuntoon 1 korostaaksesi, kuinka Cognee pitää erilliset transkriptiot samalla hyödyntäen jaettua tietämyskarttaa.


In [ ]:
await show_history(session_2)

## Yhteenveto

Onnittelut! Olet juuri antanut koodausavustajallesi pitkäaikaisen muistikerroksen Cogneen avulla.

Tässä opetusohjelmassa otit raakaa kehittäjäsisältöä (koodia, dokumentaatiota, keskusteluja) ja muutit sen graafiksi + vektorimuistiksi, jota agenttisi voi hakea, käyttää päättelyyn ja parantaa jatkuvasti.

Mitä olet oppinut

1. **Raakatekstistä tekoälymuistiin**: Kuinka Cognee käsittelee jäsentämätöntä dataa ja muuttaa sen älykkääksi, haettavaksi muistiksi yhdistetyn vektori- ja tietograafiarkkitehtuurin avulla.

2. **Graafin rikastaminen memifyllä**: Kuinka mennä perusgraafin luomisen yli ja käyttää memifyä lisättyjen faktojen ja rikkaampien suhteiden lisäämiseen olemassa olevaan graafiisi.

3. **Useita hakustrategioita**: Kuinka tehdä hakuja muistista eri hakutyypeillä (graafitietoinen kysymys-vastaus, RAG-tyylinen täydennys, oivallukset, raakapalaset, koodihaku jne.) riippuen siitä, mitä agenttisi tarvitsee.

4. **Visuaalinen tutkiminen**: Kuinka tarkastella ja debugata Cogneen luomaa sisältöä graafivisualisointien ja Cogneen käyttöliittymän avulla, jotta voit nähdä, miten tieto on jäsennelty.

5. **Istuntokohtainen muisti**: Kuinka yhdistää istuntokohtainen konteksti pysyvään semanttiseen muistiin, jotta agentit voivat muistaa eri käyttökertojen välillä ilman, että tietoa vuotaa käyttäjien välillä.


## Keskeiset huomiot
1. Muisti tietämyskarttana, jota tukevat upotukset

    - **Jäsennelty ymmärrys**: Cognee yhdistää vektorivaraston ja graafivaraston, joten tietosi ovat sekä merkityksen mukaan haettavissa että suhteiden kautta yhdistettyjä. Cognee käyttää oletuksena tiedostopohjaisia tietokantoja (LanceDB vektorivarastolle, Kuzu graafitietokannalle).

    - **Suhteet huomioiva haku**: Vastaukset voivat perustua paitsi "samankaltaiseen tekstiin", myös siihen, miten entiteetit liittyvät toisiinsa.

    - **Elävä muisti**: Muistikerros kehittyy, kasvaa ja pysyy kyseltävissä yhtenä yhdistettynä graafina.

2. Haku- ja päättelytilat
    - **Hybridi haku**: Haku yhdistää vektorien samankaltaisuuden, graafirakenteen ja LLM-päättelyn, alkaen raakadatapalojen hausta graafitietoiseen kysymysten vastaamiseen.

    - **Sovita tila tehtävään**: Käytä täydentämistyyppisiä tiloja, kun haluat luonnollisen kielen vastauksia, ja pala-/yhteenveto-/graafitiloja, kun agenttisi tarvitsee raakakontekstia tai haluaa ohjata omaa päättelyään.

3. Henkilökohtaiset, istuntotietoiset agentit
    - **Istuntokonteksti + pitkäaikainen muisti**: Cognee pitää lyhytaikaisen "keskusteluketjun" kontekstin erillään pitkäikäisestä, käyttäjä- tai organisaatiotason muistista.

## Reaaliaikaiset sovellukset

1. **Vertikaaliset tekoälyagentit**

    Käytä tämän muistikirjan mallia luodaksesi alakohtaisia älykkäitä apureita, jotka toimivat Cogneen hakujen ja päättelyn ytimenä:

- **Kehittäjäapureita**: Koodin tarkastelu, tapausanalyysi ja arkkitehtuurin avustajat, jotka käyvät läpi koodia, API:ita, suunnitteludokumentteja ja tikettejä yhtenä muistigraafina.

- **Asiakaskohtaiset apurit**: Tukipalvelu- tai menestysagentit, jotka hakevat tuotteen dokumenteista, usein kysytyistä kysymyksistä, CRM-muistiinpanoista ja aiemmista tiketeistä graafitietoisen haun ja viitattujen vastausten avulla.

- **Sisäiset asiantuntija-apurit**: Politiikka-, laki- tai turvallisuusavustajat, jotka tekevät päättelyä toisiinsa liittyvien sääntöjen, ohjeiden ja historiallisten päätösten perusteella erillisten PDF-tiedostojen sijaan.

    Cognee on nimenomaan suunniteltu pysyväksi, tarkaksi muistiksi tekoälyagenteille, tarjoten elävän tietämyskartan, joka toimii agenttisi taustalla ja korvaa ad hoc -yhdistelmät vektorivarastoista ja mukautetusta graafikoodista.

2. **Tietosiilojen yhdistäminen yhdeksi muistiksi**

    Sama lähestymistapa auttaa sinua rakentamaan yhtenäisen muistikerroksen hajallaan olevista lähteistä:

- **Siiloista yhdeksi graafiksi**: Tuo jäsenneltyä (esim. tietokannat) ja jäsentämätöntä dataa (esim. dokumentit, keskustelut) yhteen graafiin, jota tukevat upotukset, sen sijaan että jokaiselle järjestelmälle olisi erilliset indeksit.

- **Lähteiden välinen päättely viitteiden kanssa**: Suorita monivaiheista päättelyä kaiken datan yli—"yhdistä" lokit, mittarit ja dokumentit graafin kautta—ja palauta silti perustellut vastaukset lähteineen.

- **Tietokeskukset**: Esimerkiksi pankki- tai koulutusalalla Cognee yhdistää jo PDF-tiedostoja, sisäisiä järjestelmiä ja sovellustietoja yhdeksi tietämyskartaksi vektoreilla, jotta agentit voivat vastata kysymyksiin tarkalla, viitatuilla kontekstilla.

## Seuraavat askeleet

Olet toteuttanut muistiloopin ytimen. Tässä on luonnollisia jatkotoimia, joita voit kokeilla itse (katso [Cogneen dokumentaatio](https://docs.cognee.ai/) saadaksesi lisätietoja):

1. **Kokeile ajallista tietoisuutta**: Ota käyttöön ajallinen kognitio, jotta voit poimia tapahtumia ja aikaleimoja tekstistä.

2. **Ota käyttöön ontologiaohjattu päättely**: Määrittele OWL-ontologia alallesi. Käytä Cogneen ontologiatukea, jotta poimitut entiteetit ja suhteet perustuvat kyseiseen skeemaan, parantaen graafin laatua ja alakohtaisia vastauksia.

3. **Lisää palautesilmukka**: Anna Cogneen säätää graafin reunapainoja todellisen käyttäjäpalautteen perusteella, jotta haku paranee ajan myötä eikä pysy staattisena.

4. **Hienosäädä henkilökohtaistamista ja istuntokäyttäytymistä**: Käytä käyttäjätunnuksia, vuokralaisia ja tietojoukkoja antaaksesi jokaiselle henkilölle tai tiimille oman näkymän jaetun muistimoottorin yli.

5. **Laajenna monimutkaisempiin agenteihin**: Liitä Cognee agenttikehyksiin rakentaaksesi monen agentin järjestelmiä, jotka kaikki jakavat saman muistikerroksen. *Microsoft Agent Framework x Cognee -lisäosa on tulossa pian.*


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Vastuuvapauslauseke**:  
Tämä asiakirja on käännetty käyttämällä tekoälypohjaista käännöspalvelua [Co-op Translator](https://github.com/Azure/co-op-translator). Vaikka pyrimme tarkkuuteen, huomioithan, että automaattiset käännökset voivat sisältää virheitä tai epätarkkuuksia. Alkuperäinen asiakirja sen alkuperäisellä kielellä tulisi pitää ensisijaisena lähteenä. Kriittisen tiedon osalta suositellaan ammattimaista ihmiskäännöstä. Emme ole vastuussa väärinkäsityksistä tai virhetulkinnoista, jotka johtuvat tämän käännöksen käytöstä.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
